# Setup

In [ ]:
import os
import pandas as pd
import numpy as np

import json
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datasets import Dataset

# Text Similarity
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

import torch
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util

# Statistical Packages
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import mahalanobis
#from sklearn.preprocessing import StandardScaler
#from numpy.linalg import inv, pinv

# Load Inference/Simulated Data

In [ ]:
df_Inferenced_TheresaMay_llama_finetuned = pd.read_csv('/kaggle/input/inferencing-llam3-2-finetuned-theresamay-v2-ipynb/df_pairs_TheresaMay_simulated_Llama3.2_finetuned.csv')
df_Inferenced_TheresaMay_llama_base = pd.read_csv('/kaggle/input/inferencing-llam3-2-base-theresamay-v2-ipynb/df_pairs_TheresaMay_simulated_Llama3.2_base.csv')
df_Inferenced_TheresaMay_gpt = pd.read_csv('/kaggle/input/inferencing-gpt3-5-theresamay-ipynb/df_pairs_TheresaMay_simulated_Gpt3.5.csv')

In [ ]:
df_Inferenced_TheresaMay_llama_finetuned.columns

## Define Similarity Metrics

Sentence Embeddings (BERT, RoBERTa, Sentence-BERT)
- BERT (Bidirectional Encoder Representations from Transformers): A transformer-based model that provides contextual embeddings for words and sentences.
- RoBERTa: A robustly optimized version of BERT.
- Sentence-BERT: A modification of BERT that produces semantically meaningful sentence embeddings.

In [ ]:
## BLEU Scores
nltk.download('punkt')
def compute_bleu(reference, candidate):
    reference = [nltk.word_tokenize(reference.lower())]
    candidate = nltk.word_tokenize(candidate.lower())
    smoothie = SmoothingFunction().method4  # Avoid zero BLEU for short sentences
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)

## ROUGE Scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
def compute_rouge(reference, candidate):
    scores = scorer.score(reference, candidate)
    return scores['rouge1'].fmeasure

# Sentence Embeddings Simlarity
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def compute_cosine_similarity(reference, candidate):
    ref_embedding = embedding_model.encode(reference, convert_to_tensor=True)
    cand_embedding = embedding_model.encode(candidate, convert_to_tensor=True)
    return util.pytorch_cos_sim(ref_embedding, cand_embedding).item()


## Compute & Plot

In [ ]:
# Compute similarity metrics for all models
for df, model_name in zip([df_Inferenced_TheresaMay_llama_finetuned, 
                           df_Inferenced_TheresaMay_llama_base, 
                           df_Inferenced_TheresaMay_gpt], 
                          ['Llama3.2-Finetuned', 'Llama3.2-Base', 'GPT-3.5']):
    df['BLEU'] = df.apply(lambda row: compute_bleu(row['response'], row['model_response']), axis=1)
    df['ROUGE'] = df.apply(lambda row: compute_rouge(row['response'], row['model_response']), axis=1)
    df['Cosine_Similarity'] = df.apply(lambda row: compute_cosine_similarity(row['response'], row['model_response']), axis=1)
    df['Model'] = model_name  # Add a column to identify the model

In [ ]:
# Combine all results
df_plotting_TM = pd.concat([
    df_Inferenced_TheresaMay_llama_finetuned[['BLEU', 'ROUGE', 'Cosine_Similarity', 'Model']],
    df_Inferenced_TheresaMay_llama_base[['BLEU', 'ROUGE', 'Cosine_Similarity', 'Model']],
    df_Inferenced_TheresaMay_gpt[['BLEU', 'ROUGE', 'Cosine_Similarity', 'Model']]
])

df_plotting_melted_TM = df_plotting_TM.melt(id_vars=['Model'], var_name='Metric', value_name='Score')
df_plotting_melted_TM['Metric'] = df_plotting_melted_TM['Metric'].replace('Cosine_Similarity', 'Sentence Embeddings Simlarity')

In [ ]:
# Plot the box plots
plt.figure(figsize=(14, 8))
sns.boxplot(x='Metric', y='Score', hue='Model', 
            data=df_plotting_melted_TM, palette="Set2", width=0.6)
plt.title('Model Similarity - Real vs. Simulated Response (TheresaMay)', fontsize=18, fontweight='bold')
plt.xlabel('Metric', fontsize=16)
plt.ylabel('Score', fontsize=16)
plt.xticks(#rotation=0, ha='right', 
           fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title='Model', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.ylim(0,1)

plt.savefig('/kaggle/working/models_similarity_TheresaMay')
plt.show()

# Load LIWC Results Data

In [ ]:
# Define the columns to exclude
exclude_columns = ['prompt_date', 'prompt_agenda', 'prompt_speaker', 'prompt', 'prompt_terms', 'response_terms', 'Text']

# Define a lambda function to filter out the columns to exclude
usecols = lambda column: column not in exclude_columns

# Load the dataframes with the filtered columns
LIWC_df_TheresaMay_gpt = pd.read_csv('/kaggle/input/debate-results/LIWC_analysis_TheresaMay_gpt.csv', usecols=usecols)
LIWC_df_TheresaMay_llamaBase = pd.read_csv('/kaggle/input/debate-results/LIWC_analysis_TheresaMay_llama_base.csv', usecols=usecols)
LIWC_df_TheresaMay_llamaTuned = pd.read_csv('/kaggle/input/debate-results/LIWC_analysis_TheresaMay_llama_finetuned.csv', usecols=usecols)

print(LIWC_df_TheresaMay_gpt.columns)

In [ ]:
# Check for NAs
na_counts_gpt = LIWC_df_TheresaMay_gpt.isna().sum()
na_counts_llamaBase = LIWC_df_TheresaMay_llamaBase.isna().sum()
na_counts_llamaTuned = LIWC_df_TheresaMay_llamaTuned.isna().sum()

# Print columns with NAs
print("Columns with NAs in LIWC_df_TheresaMay_gpt:")
print(na_counts_gpt[na_counts_gpt > 0])

print("\nColumns with NAs in LIWC_df_TheresaMay_llamaBase:")
print(na_counts_llamaBase[na_counts_llamaBase > 0])

print("\nColumns with NAs in LIWC_df_TheresaMay_llamaTuned:")
print(na_counts_llamaTuned[na_counts_llamaTuned > 0])

In [ ]:
# Replace NA values with 0 
LIWC_df_TheresaMay_gpt = LIWC_df_TheresaMay_gpt.fillna(0)
LIWC_df_TheresaMay_llamaBase = LIWC_df_TheresaMay_llamaBase.fillna(0)
LIWC_df_TheresaMay_llamaTuned = LIWC_df_TheresaMay_llamaTuned.fillna(0)

In [ ]:
liwc_features = LIWC_df_TheresaMay_llamaTuned.columns[3:,]
liwc_features

In [ ]:
liwc_categories = {
    "Summary Variables": ['Analytic', 'Clout', 'Authentic', 
                          'Tone', 'WPS', 'BigWords', 'Dic'],
    "Linguistic Dimensions": ['Linguistic', 'function', 'pronoun', 'ppron', 'ipron', 'det', 
                              'article', 'number', 'prep', 'auxverb', 'adverb', 'conj', 
                              'negate', 'verb', 'adj', 'quantity'],
    "Psychological Processes": ['Drives', 'affiliation', 'achieve', 'power', 'Cognition', 
                                'allnone', 'cogproc', 'insight', 'cause', 'discrep', 'tentat', 
                                'certitude', 'differ', 'memory', 'Affect', 'tone_pos', 'tone_neg', 
                                'emoti,on', 'emo_pos', 'emo_neg', 'emo,anx', 'emo_anger', 'emo,sad', 
                                'socbehav', 'pro,social', 'polite', 'confl,ict', 'moral', 'soc,refs', 
                                'family', 'friend', 'female', 'male'],
    'Culture': ['Culture', 'politic', 'ethnicity', 'tech'],
    'Lifestyle': ['lifestyle', 'leisure', 'home', 'work', 'money', 'relig'],
    'Physical': ['physical', 'health', 'illness', 'wellness', 'mental', 'substances', 'sexual', 'food', 'death'],
    'States': ['need', 'want', 'acquire', 'lack', 'fulfill', 'fatigue'],
    'Motives': ['reward', 'risk', 'curiosity', 'allure'],
    'Perception': ['Perception', 'attention', 'motion', 'space', 'visual', 'auditory', 'feeling']
}

In [ ]:
models_TM = {
    "GPT-3.5": LIWC_df_TheresaMay_gpt,
    "Llama Base": LIWC_df_TheresaMay_llamaBase,
    "Llama Fine-Tuned": LIWC_df_TheresaMay_llamaTuned
}

## LIWC Cosine Similarity

In [ ]:
cosine_similarity_results_TM = []

for model_name, df_model in models_TM.items():
    for category, features in liwc_categories.items():

        features = [f for f in features if f in df_model.columns]
        
        if not features:  # Skip categories without valid features
            continue

        # Compute Cosine Similarity for each pair of consecutive rows
        for i in range(0, len(df_model) - 1, 2):
            real_response = df_model.iloc[i][features].values.reshape(1, -1)
            model_response = df_model.iloc[i+1][features].values.reshape(1, -1)
            
            cos_sim = cosine_similarity(real_response, model_response)[0][0]
            cosine_similarity_results_TM.append({
                "Model": model_name,
                "Category": category,
                "Cosine Similarity": cos_sim
            })

df_LIWC_cosine_sim_TM = pd.DataFrame(cosine_similarity_results_TM)
df_LIWC_cosine_sim_filtered_TM = df_LIWC_cosine_sim_TM[~df_LIWC_cosine_sim_TM['Category'].isin(["Culture", "Lifestyle", "Physical", "States"])]

plt.figure(figsize=(14, 8))
sns.boxplot(data = df_LIWC_cosine_sim_filtered_TM, 
            x="Category", y="Cosine Similarity", 
            hue="Model", palette="coolwarm", width=0.6)
plt.title("Linguistic Feature Alignment of LLM Models: LIWC Cosine Similarity", fontsize=18, fontweight='bold')
plt.ylabel("Cosine Similarity", fontsize=14)
plt.xlabel("LIWC Categories", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title="Model", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('/kaggle/working/models_similarity_LIWC_cosine_TheresaMay.png')
plt.show()

## LIWC Mahalanobis Distance

In [ ]:
mahalanobis_results_TM = []

for model_name, df_model in models_TM.items():
    for category, features in liwc_categories.items():
        # Ensure features exist in the dataset
        features = [f for f in features if f in df_model.columns]
        
        if not features:  # Skip categories without valid features
            continue

        # Compute Mahalanobis Distance for each pair of consecutive rows
        for i in range(0, len(df_model) - 1, 2):
            real_response = df_model.iloc[i][features].values
            model_response = df_model.iloc[i+1][features].values

            # Compute covariance matrix and inverse
            cov_matrix = np.cov(df_model[features].dropna().T)
            inv_cov_matrix = np.linalg.pinv(cov_matrix)  # Use pseudo-inverse for numerical stability

            # Compute Mahalanobis distance
            mahal_dist = mahalanobis(real_response, model_response, inv_cov_matrix)
            mahalanobis_results_TM.append({
                "Model": model_name,
                "Category": category,
                "Mahalanobis Distance": mahal_dist
            })

df_LIWC_mahalanobis_TM = pd.DataFrame(mahalanobis_results_TM)
df_LIWC_mahalanobis_filtered_TM = df_LIWC_mahalanobis_TM[~df_LIWC_mahalanobis_TM['Category'].isin(["Culture", "Lifestyle", "Physical", "States"])]

plt.figure(figsize=(14, 8))
sns.boxplot(data = df_LIWC_mahalanobis_filtered_TM, 
            x="Category", y="Mahalanobis Distance", 
            hue="Model", palette="coolwarm", width=0.6)
plt.title("Linguistic Feature Alignment of LLM Models: LIWC Mahalanobis Distance", fontsize=18, fontweight='bold')
plt.ylabel("Mahalanobis Distance", fontsize=14)
plt.xlabel("LIWC Categories", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title="Model", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('/kaggle/working/models_similarity_LIWC_mahalanobis_Theresa May.png')
plt.show()